# Assignment

-------------

**1) Complete this notebook and make a pull request:** 

Answer questions (Q) in the space provided (A) in this notebook. When finished, copy your notebook to the `Assignment/` directory and name it `nb-6.5-<Github-username>.ipynb`. Then make a pull request to the upstream repo. The entered answers in this notebook will be simply Markdown text where I want you to interpret and describe a block of code to better understand what it is doing. Much of this code you will have seen already. 


**2) Write an importable Python package, save as a repo, and test it here.**

The package should be written as we did in our last lession (`.py` files in a directory with a setup.py file so it can be installed with `pip`). Follow instructions at the end of this notebook for how to write your package. Test it here by importing the package and executing the code at the end. It should work and give correct answers, if not, continue working on it. When you have it completed save your package as a new Github repo named `seqlib`.

### The `seqlib` package

Together we are going to write several functions here that will make up your new package called `seqlib`. It will be your job to copy these functions, organize them into a Class, save the code into a `.py` file (you can use SublimeText if you're comfortable with it for much of this, or any text editor including the one in jupyter), package the files so they can be imported as a library, and test the package so that it accomplishes the tasks which are defined at the end of this notebook. First things first, though, let's write the functions. 

In [1]:
import numpy as np
import pandas as pd

### Q.  Describe what the `mutate()` function below does:


A. Remove the input base from the set of "ACTG" and then return a random base from the resulting set. This will return any base other than the one that was input.

In [8]:
def mutate(base):
    diff = set("ACTG") - set(base)
    return np.random.choice(list(diff))

In [15]:
# test it
mutate("A")

'G'

### Q. Describe how the `seqdata()` function below works:
Annotate the code by inserting lines with comments as you read through the function to make sense of it. What is being created at each step and how is it used?


A. See my lines with `#` below.

In [16]:
# create a function given two arguments (rows and columns) to simulate a DNA sequence
def simulate(ninds, nsites):
    
    # define a random sequence of bases of nsites length
    oseq = np.random.choice(list("ACGT"), size=nsites)
    
    # create array with above-defined oseq by ninds number of columns
    arr = np.array([oseq for i in range(ninds)])
    
    # create random mutation probabilities
    muts = np.random.binomial(1, 0.1, (ninds, nsites))
    
    # iterate over each column
    for col in range(nsites):
        
        # perform the mutate function
        newbase = mutate(arr[0, col])
        
        # boolean mask to select by mutation probability
        mask = muts[:, col].astype(bool)
        
        # assign the same mutation to successive iterations in the same column
        arr[:, col][mask] = newbase
        
    # assign random probability for each point in the array
    missing = np.random.binomial(1, 0.1, (ninds, nsites))
    
    # code "missing" data as "N"
    arr[missing.astype(bool)] = "N"
    
    # return the array resulting from the above code
    return arr

In [17]:
seqs = simulate(6, 15)
print(seqs)

[['A' 'C' 'A' 'N' 'G' 'C' 'N' 'T' 'A' 'C' 'A' 'N' 'A' 'A' 'G']
 ['A' 'C' 'C' 'C' 'G' 'C' 'G' 'G' 'A' 'C' 'A' 'C' 'A' 'A' 'G']
 ['A' 'N' 'C' 'C' 'N' 'C' 'G' 'N' 'A' 'C' 'A' 'C' 'A' 'N' 'T']
 ['A' 'C' 'C' 'N' 'G' 'N' 'G' 'G' 'A' 'C' 'A' 'C' 'A' 'A' 'G']
 ['C' 'N' 'C' 'C' 'G' 'A' 'G' 'T' 'A' 'C' 'A' 'C' 'A' 'A' 'G']
 ['A' 'C' 'C' 'C' 'G' 'C' 'G' 'G' 'A' 'C' 'A' 'C' 'A' 'A' 'G']]


### **Q: Describe how the `filter_missing` function works:**
Annotate the code by inserting lines with comments as you read through the function to make sense of it. How does it find columns with missing (N) values in them? How might you mprove it?

A. See my lines with `#` below.

In [6]:
# define a function given two arguments (an array and a maximum frequency)
def filter_missing(arr, maxfreq):
    
    # define the frequency of missing values as the sum of the number of times that the array has "N" divided by the shape value
    freqmissing = np.sum(arr == "N", axis=0) / arr.shape[0]
    
    # create an array with only the columns that have a frequency of missing values less than or equal to the input maximum frequency
    # aka an array of only the columns with no missing data 
    return arr[:, freqmissing <= maxfreq]

In [7]:
filter_missing(seqs, 0.1)

array([['G', 'G', 'T', 'A', 'C', 'C', 'G', 'G'],
       ['G', 'G', 'G', 'A', 'C', 'C', 'A', 'G'],
       ['G', 'T', 'T', 'C', 'C', 'C', 'G', 'G'],
       ['G', 'G', 'T', 'A', 'C', 'C', 'A', 'A'],
       ['G', 'G', 'T', 'A', 'C', 'C', 'G', 'G'],
       ['G', 'G', 'T', 'A', 'C', 'C', 'G', 'G']],
      dtype='<U1')

### **Q: Describe how the `filter_maf` function works:**
Annotate the code by inserting lines with comments as you read through the function to make sense of it. How does it calculate minor allele frequencies? Why does it use copy?

A. See my lines with `#` below.

In [8]:
# coming back to this...
def filter_maf(arr, minfreq):
    
    #
    freqs = np.sum(arr != arr[0], axis=0) / arr.shape[0]
    
    #
    maf = freqs.copy()
    
    #
    maf[maf > 0.5] = 1 - maf[maf > 0.5]
    
    #
    return arr[:, maf > minfreq]

In [9]:
filter_maf(seqs, 0.1)

array([['G', 'T', 'A', 'T', 'N', 'G', 'N', 'C', 'G', 'G', 'A', 'T'],
       ['G', 'G', 'A', 'T', 'G', 'G', 'N', 'C', 'A', 'G', 'A', 'T'],
       ['T', 'T', 'C', 'N', 'G', 'N', 'N', 'G', 'G', 'G', 'G', 'T'],
       ['G', 'T', 'A', 'T', 'G', 'G', 'G', 'C', 'A', 'A', 'A', 'C'],
       ['G', 'T', 'A', 'T', 'N', 'G', 'G', 'N', 'G', 'G', 'G', 'N'],
       ['G', 'T', 'A', 'T', 'G', 'G', 'T', 'C', 'G', 'G', 'N', 'C']],
      dtype='<U1')

### Q: What order should these functions be applied, does it matter?

A. The output is the same for both orders.

In [10]:
filter_missing(filter_maf(seqs, 0.1), 0.1)

array([['G', 'T', 'A', 'G', 'G'],
       ['G', 'G', 'A', 'A', 'G'],
       ['T', 'T', 'C', 'G', 'G'],
       ['G', 'T', 'A', 'A', 'A'],
       ['G', 'T', 'A', 'G', 'G'],
       ['G', 'T', 'A', 'G', 'G']],
      dtype='<U1')

In [11]:
filter_maf(filter_missing(seqs, 0.1), 0.1)

array([['G', 'T', 'A', 'G', 'G'],
       ['G', 'G', 'A', 'A', 'G'],
       ['T', 'T', 'C', 'G', 'G'],
       ['G', 'T', 'A', 'A', 'A'],
       ['G', 'T', 'A', 'G', 'G'],
       ['G', 'T', 'A', 'G', 'G']],
      dtype='<U1')

### Q: Describe how `calculate_statistics()` works


A. `calculate_statistics()` uses numpy to generate statistical values and shape them into an array, and then uses panda to apply labels to display with the statistical values.

In [12]:
def calculcate_statistics(arr):
    nd = np.var(arr == arr[0], axis=0).mean()
    mf = np.mean(np.sum(arr != arr[0], axis=0) / arr.shape[0])
    inv = np.any(arr != arr[0], axis=0).sum()
    var = arr.shape[1] - inv
    return pd.Series(
        {"mean nucleotide diversity": nd,
         "mean minor allele frequency": mf,
         "invariant sites": inv,
         "variable sites": var,
        })

In [13]:
calculcate_statistics(seqs)

invariant sites                12.000000
mean minor allele frequency     0.255556
mean nucleotide diversity       0.150000
variable sites                  3.000000
dtype: float64

### Instructions: Write a `seqlib` Class object

I started writing the bare bones of it below. You should write it so that it can be executed as described below to perform all of the functions we defined above, and so that its attributes can be accessed. Save this class object in a `.py` file and make it into an importable package called `seqlib`. You can write and test your object in this notebook if you like, but it must be saved separately in a `.py` file and be imported. You cannot execute the code at the end using your object defined here in the notebook. When finished save your package to GitHub as a repo just like we did with the `helloworld` package. You do not need to write a CLI script like we did for the `helloworld` package, we will only be using the Python API here. See the examples below for **how you should write your Class object**. It should be able to run in the way written below, so look at that code and think about how you would write a Class object that can do that. 

While you can mostly copy the functions from above, you will need to modify them slightly to access information about the Class object using *self*. For example, the `simulate()` function below takes self as a first argument and can access `self.inds` and `self.nsites` from that, so we do not need to provide those as arguments to the `simulate` function like we did above. 

In [ ]:
# to be continued...
class Seqlib:
    def __init__(self, ninds, nsites):
        self.ninds = ninds
        self.nsites = nsites
        self.seqs = self.simulate()
        # ...
        
    def simulate(self):
        pass
        # ...
        
    # continue writing this full object..
    

## Test your package
The package should be globally importable (you ran `pip install .` or `pip install -e .` to install it), and it should be able to execute the following code without error. 

In [ ]:
import seqlib

In [ ]:
# init a Seqlib Class object
seqs = seqlib.Seqlib(ninds=10, nsites=50)

In [ ]:
# access attributes from the object
print(seqs.ninds, seqs.nsites)

In [ ]:
# returns the MAF of the array as an array of floats
seqs.maf

In [ ]:
# return a view of the filtered sequence array by applying a new function 
# called `filter()` that applies both the maf and missing filter functions
seqs.filter(minmaf=0.1, maxmissing=0.0)

In [ ]:
# calculate statistics for an array with the results returned as a DataFrame
seqs.calculate_statistics()

In [ ]:
# calculate statistics for an array after filtering it
seqs.filter(minmaf=0.1, maxmissing=0.0).calculate_statistics()